It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1399 in data/validation/cats
- put the dogs pictures index 0-999 in data/train/dogs
- put the dog pictures index 1000-1399 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        
        cats/
            cat.0.jpg
            cat.1.jpg
            ...
        dogs/
            dog.0.jpg
            dog.1.jpg
            ...  
    validation/
        cats/
            cat.1000.jpg
            cat.1001.jpg
            ...
        dogs/
            dog.1000.jpg
            dog.1001.jpg
            ...

In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

In [3]:
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [4]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.vgg16.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

In [5]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array([0] * 1000 + [1] * 1000)

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array([0] * 400 + [1] * 400)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [6]:
save_bottlebeck_features()
train_top_model()

Found 2000 images belonging to 2 classes.


C:\Users\boshi\AppData\Local\Temp/ipykernel_7360/3673972680.py:13: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_train = model.predict_generator(


Found 23000 images belonging to 2 classes.


C:\Users\boshi\AppData\Local\Temp/ipykernel_7360/3673972680.py:24: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_validation = model.predict_generator(


Epoch 1/50
125/125 [==============================] - 1s 5ms/step - loss: 0.6506 - accuracy: 0.7755 - val_loss: 1.4565 - val_accuracy: 0.4913
Epoch 2/50
125/125 [==============================] - 1s 5ms/step - loss: 0.3512 - accuracy: 0.8560 - val_loss: 1.4605 - val_accuracy: 0.4913
Epoch 3/50
125/125 [==============================] - 1s 5ms/step - loss: 0.3009 - accuracy: 0.8790 - val_loss: 2.8835 - val_accuracy: 0.4988
Epoch 4/50
125/125 [==============================] - 1s 5ms/step - loss: 0.2522 - accuracy: 0.9015 - val_loss: 2.3581 - val_accuracy: 0.4863
Epoch 5/50
125/125 [==============================] - 1s 5ms/step - loss: 0.2274 - accuracy: 0.9085 - val_loss: 3.4225 - val_accuracy: 0.4863
Epoch 6/50
125/125 [==============================] - 1s 4ms/step - loss: 0.2036 - accuracy: 0.9230 - val_loss: 3.3492 - val_accuracy: 0.4812
Epoch 7/50
125/125 [==============================] - 1s 5ms/step - loss: 0.1776 - accuracy: 0.9355 - val_loss: 3.2560 - val_accuracy: 0.4800
Epoch 